In [1]:
import numpy as np
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
from theano import scan, function
%matplotlib inline

In [ ]:
# compute tanh(XW + b)

X, W = T.matrices("X", "W")
b_sym = T.vector("b_sym")

results, updates = theano.scan(lambda v: T.tanh(T.dot(v, W) + b_sym), sequences=X)
compute_elementwise = theano.function(inputs=[X,W,b_sym], outputs=results)

x = np.eye(2)
w = np.ones((2, 2))
b = np.ones((2))
b[1] = 2

print(compute_elementwise(x, w, b))
print(np.tanh(x.dot(w) + b))

In [ ]:
# compute sequence x(t) = tanh(x(t-1).dot(W) + y(t).dot(U) + p(T-t).dot(V))
X, b_sym = T.vectors("X", "b_sym")
W, U, Y, V, P = T.matrices("W", "U", "Y", "V", "P")

results, updates = theano.scan(lambda y, p, x_tm1: T.tanh(T.dot(x_tm1, W) + T.dot(y, U) + T.dot(p, V)), 
                              sequences = [Y, P[::-1]], outputs_info=[X])
compute_seq = theano.function(inputs=[X, W, Y, U, P, V], outputs=results)

# test values
x = np.zeros(2)
x[1] = 1
w = np.ones((2, 2))
y = np.ones((5, 2))
y[0, :] = -3
u = np.ones((2, 2))
p = np.ones((5, 2))
p[0, :] = 3
v = np.ones((2,2))

print(compute_seq(x, w, y, u, p, v))

In [ ]:
# Compute Eucliedian Norms of rows and columns of X

X = T.matrix("X")
results, updates = theano.scan(lambda x_i: T.sqrt((x_i**2).sum()), sequences=[X])
compute_norm_rows = theano.function(inputs=[X], outputs=results)

# test values
x = np.diag(np.arange(1, 6), 1)
a = np.diag(np.arange(1, 6), -1)
print(compute_norm_rows(x))

res_col, updates_col = theano.scan(lambda x_i: T.sqrt((x_i**2).sum()), sequences=[X.T])
compute_norm_cols = theano.function([X], res_col, updates=updates_col)

print(compute_norm_cols(x))

In [ ]:
# compute a distance matrix of classic XOR data
MU = [(0, 0), (1, 1), (0, 1), (1, 0)]
sigma = 0.2**2
N = 400
m = len(MU[0])

x1 = [np.random.multivariate_normal(MU[a], np.diag(np.repeat(sigma, m))) for a in (0 if np.random.rand() > 0.5 else 1 for i in range(N))]
x2 = [np.random.multivariate_normal(MU[a], np.diag(np.repeat(sigma, m))) for a in (2 if np.random.rand() > 0.5 else 3 for i in range(N))]
x = np.r_[x1, x2]
viz_pts = False
if viz_pts:
    plt.scatter(*np.array(x1).T, c="r")
    plt.scatter(*np.array(x2).T, c="b")
    plt.show()

# Sklearn 
from sklearn.metrics.pairwise import euclidean_distances
print("Sklearn:")
%time D_sk = euclidean_distances(x)
print(D_sk.shape)

# Numpy Way
def f_np(X):
    D = np.empty((X.shape[0], X.shape[0]))
    for i in range(X.shape[0]):
        D[i,:] = np.sqrt(((X[i] - X)**2).sum(axis=1)) 
    return(D)
print("Pure Numpy:")
%time D_np = f_np(x)
print(np.allclose(D_sk, D_np))

# Theano Way

X = T.matrix("X")
i = T.iscalar("i")

# shit way without scan
X_diff = T.sqrt(((X[i] - X)**2).sum(axis=1))
f = theano.function([X, i], X_diff)

print("Hybrid Theano/Numpy:")
%time D_th0 = np.array([f(x, i)for i in range(x.shape[0])])
print(np.allclose(D_sk, D_th0))

# pure Theano code
res, ups = theano.scan(lambda i: T.sqrt(((X[i] - X)**2).sum(axis=1)), 
                       sequences=T.arange(X.shape[0]), outputs_info=None)
f_scan = theano.function([X], res)

print("Pure Theano:")
%time D_th1 = f_scan(x)
print(np.allclose(D_sk, D_th1))


In [68]:
# Computing the trace of a matrix

X = T.matrix("X")
res, ups = theano.scan(lambda i, j, t_f: T.cast(X[i, j] + t_f, dtype="float32"), # dtype is required here
                       sequences=[T.arange(X.shape[0]), T.arange(X.shape[1])], 
                       outputs_info=np.asarray(0., dtype="float32"))
res = res[-1]
compute_trace = theano.function([X], res) # , updates=ups
x = np.diag(np.arange(5))
print(x)
print(compute_trace(x))

[[0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 2 0 0]
 [0 0 0 3 0]
 [0 0 0 0 4]]
10.0


In [6]:
# Scan Example "Time-Series"
X, W, U, V = T.matrices("X", "W", "U", "V")
b_sym = T.vector("b_sym") # b_sym = symbolic b
n_sym = T.iscalar("n_sym")

res, ups = scan(lambda x_tm2, x_tm1: T.dot(x_tm2, U) + T.dot(x_tm1, V) + T.tanh(T.dot(x_tm1, W) + b_sym), 
                n_steps=n_sym, outputs_info=[{"initial":X, "taps":[-2, -1]}])
compute_seq2 = function([X, U, V, W, b_sym, n_sym], res)

# test values

x = np.zeros((2,2))
x[1, 1] = 1
w = 0.5 * np.ones((2,2))
u = 0.5 * np.eye(2)[::-1]
v = 0.5 * np.ones((2,2))
n = 10
b = np.ones(2)

print(compute_seq2(x, u, v, w, b, n))


[[  1.40514825   1.40514825]
 [  2.88898899   2.38898899]
 [  4.34018291   4.34018291]
 [  6.53463142   6.78463142]
 [  9.82972243   9.82972243]
 [ 14.22203814  14.09703814]
 [ 20.07439936  20.07439936]
 [ 28.12291843  28.18541843]
 [ 39.1913681   39.1913681 ]
 [ 54.28407732  54.25282732]]


In [19]:
# Compute the Jacobian of y = tanh(v.dot(A)) wrt x

v = T.vector()
A = T.matrix()
y = T.tanh(T.dot(v, A))
res, ups = scan(lambda i: T.grad(y[i], v), sequences=T.arange(y.shape[0]))
compute_jac_t = function([A, v], res, allow_input_downcast=True)

# test values
x = np.eye(5)[0]
w = np.eye(5, 3)
w[2] = np.ones(3)
print(x)
print(w)
print(np.tanh(np.dot(x, w)))
print(compute_jac_t(w, x))


[ 1.  0.  0.  0.  0.]
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 1.  1.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[ 0.76159416  0.          0.        ]
[[ 0.41997434  0.          0.41997434  0.          0.        ]
 [ 0.          1.          1.          0.          0.        ]
 [ 0.          0.          1.          0.          0.        ]]


In [40]:
# Compute/Capture number of iterations

k = theano.shared(0)
n_sym = T.iscalar()
step = T.iscalar() # added variable step size

res, ups = scan(lambda step:{k:(k+step)}, n_steps=n_sym, non_sequences=[step]) # added step to lambda and non_seq
accumulator = function([n_sym, step], [], updates=ups, allow_input_downcast=True)

print(k.get_value())
accumulator(10, 2)
print(k.get_value())

0
20


In [46]:
# Computing tanh(v.dot(W) + b) * d where d is binomial

X, W = T.matrices("X", "W")
b_sym = T.vector("b_sym")

# define shared random state
trng = T.shared_randomstreams.RandomStreams(1234)
d = trng.binomial(size=W[1].shape)

res, ups = scan(lambda v: T.tanh(T.dot(v, W) + b_sym) * d, sequences=X)
compute_with_bnoise = function([X, W, b_sym], res, updates=ups, allow_input_downcast=True)

x = np.eye(10, 2)
w = np.ones((2, 2))
b = np.ones(2)
print(x)
print(compute_with_bnoise(x, w, b))

[[ 1.  0.]
 [ 0.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]
[[ 0.96402758  0.        ]
 [ 0.          0.96402758]
 [ 0.          0.        ]
 [ 0.76159416  0.76159416]
 [ 0.76159416  0.        ]
 [ 0.          0.76159416]
 [ 0.          0.76159416]
 [ 0.          0.76159416]
 [ 0.          0.        ]
 [ 0.76159416  0.76159416]]


In [56]:
# Compute pow(A,k)
k = T.iscalar("k")
A = T.vector("A")

def inner_fct(prior_result, B):
    return prior_result * B

# Symbolic description of the result
result, updates = theano.scan(fn=inner_fct,
                            outputs_info=T.ones_like(A),
                            non_sequences=A, n_steps=k)

# Scan has provided us with A ** 1 through A ** k.  Keep only the last
# value. Scan notices this and does not waste memory saving them.
final_result = result[-2:]

power = theano.function(inputs=[A, k], outputs=final_result,
                      updates=updates)

print(power(range(10), 3))

[[   0.    1.    4.    9.   16.   25.   36.   49.   64.   81.]
 [   0.    1.    8.   27.   64.  125.  216.  343.  512.  729.]]


In [91]:
# Calculating a Polynomial

# theano.config.warn.subtensor_merge_bug = False

coeffs = T.vector("coeffs")
x = T.scalar("x")
outputs_info = T.as_tensor_variable(np.asarray(0, 'float64')) # set to accumulate results
max_coeffs_supported = 10000

# Generate components of polynomial

full_range = T.arange(max_coeffs_supported)
components, updates = scan(lambda coeff, power, pv, free_var: pv + coeff * (free_var ** power), 
                           sequences=[coeffs, full_range], non_sequences=x, 
                           outputs_info=outputs_info) # changed from none to accumulate results
polynomial = components[-1]
calculate_polynomial = function([coeffs, x], polynomial)

# test values
test_coeffs = np.array([1, 0, 2])
print(calculate_polynomial(test_coeffs, 3))

19.0
